In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("data.csv")
data[data==' ']=np.nan
data=data.dropna()
data.head(5)

/afs/inf.ed.ac.uk/user/s18/s1883226/miniconda3/envs/mlp/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ID painter1_time painter2_time painter3_time painter4_time painter5_time  \
0  25       567.443       589.258       871.075       424.541       640.152   
1  33      1424.242        63.744       356.716       325.936       105.141   
2  42      1320.878       183.888      1147.679       806.613       569.499   
3  58       897.658       681.745       368.636       727.118       723.261   
4  68      1078.837       611.756       462.712       763.973       627.325   

  painter6_time painter7_time painter8_time painter9_time  ...  \
0      1117.124       908.755       493.497       906.011  ...   
1       951.267      1160.728       155.481       899.991  ...   
2       904.191      1507.476       294.256       393.701  ...   
3      1070.142       874.328       636.127       592.833  ...   
4       789.635       576.564       297.831       535.604  ...   

  painter10_familiarity stress tired pay_off risk b_4 b_3 c_4 c_3 delay  
0                     1      1     1     454    2   0  37   5  11     2  
1                     1      2     2     507    2   5  31  12   6     2  
2                     1      1     1     434    1   0  24   9   7     1  
3                     1      0     0     566    2  12  40   5   9     1  
4                     1      0     0     268    1   0  16  19   4     2  

[5 rows x 40 columns]

In [2]:
# np.save('data_E.npy',data_E)
# data_E=np.load('data_E.npy')
# data_E
# stress=np.int8(data_E.iloc[:,31])

In [3]:
data_E=data.iloc[0:86,:]
data_T=data.iloc[86::,:]

In [4]:
stress=np.int8(data_E.iloc[:,31])
tired=np.int8(data_E.iloc[:,32])
st=np.int8(data_E.iloc[:,31:33])
choice3=data_E.iloc[:,37]
choice4=data_E.iloc[:,38]
tend=np.int8(choice3)+np.int8(choice4)

In [5]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(st, tend, test_size=0.2, random_state=0)


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(st)))
r2_score(tend, lin_reg.predict(st))

4.912575694861349


0.003442544562985872

In [7]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(st)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, tend)
X_poly_test = poly_reg.fit_transform(st)
print(mean_absolute_error(tend, pol_reg.predict(X_poly_test)))
print(r2_score(tend, pol_reg.predict(X_poly_test)))
pol_reg.coef_

4.9420091603811676
0.01532638321958868


array([ 0.        , -0.96444428,  1.24350086,  0.90582201,  0.60012755,
       -0.97726721])

In [8]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(X_train)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y_train)
X_poly_test = poly_reg.fit_transform(st)
print(mean_absolute_error(tend, pol_reg.predict(X_poly_test)))
print(r2_score(tend, pol_reg.predict(X_poly_test)))
pol_reg.coef_

4.913692496538577
0.009243952054722948


array([ 0.        , -1.91508998,  0.66987016,  1.23326549,  0.652249  ,
       -0.58347982])

In [9]:
data_p=np.zeros((86*10,13))
data_clean=np.array(data_E)
for i in range (0,86):
    data_p[i*10:i*10+10,0]=data_clean[i,0] #ID
    data_p[i*10:i*10+10,1]=data_clean[i,1:11] #time
    data_p[i*10:i*10+10,2]=data_clean[i,11:21] #choice
    data_p[i*10:i*10+10,3]=data_clean[i,21:31] #familarity
    data_p[i*10:i*10+10,4]=data_clean[i,31] #stress
    data_p[i*10:i*10+10,5]=data_clean[i,32] #tired
    data_p[i*10:i*10+10,6]=data_clean[i,33] #pay off
    data_p[i*10:i*10+10,7]=data_clean[i,34] #risk
    data_p[i*10:i*10+10,8]=data_clean[i,35] #bounce 4
    data_p[i*10:i*10+10,9]=data_clean[i,36] #bounce 3
    data_p[i*10:i*10+10,10]=data_clean[i,37] #choice 4 include 3?
    data_p[i*10:i*10+10,11]=data_clean[i,38] #choice 3
    data_p[i*10:i*10+10,12]=data_clean[i,39] #delay

In [10]:
from distribut import best_fit_distribution

In [11]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    DISTRIBUTIONS = [st.burr]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

for i in range (1,5):
    y=data_p[np.where(data_p[:,3]==i),1]
    print(best_fit_distribution(y, bins=200, ax=None))


('burr', (3.296575094179248, 0.6095424311772768, -6.657664186104298, 818.0481187486655))
('burr', (2.96278230336626, 0.7060818040736765, 0.03808196123981357, 617.1479208748744))
('burr', (3.8794797413534, 0.39063634718617846, -3.6147574966601823, 667.2560086831902))
('burr', (5.607764646791989, 0.24090663005789292, -3.6836905712303682, 726.6320778736202))


In [12]:
for i in range (1,5):
    y=data_p[np.where(data_p[:,3]==i),1]
    m=best_fit_distribution(y, bins=200, ax=None)
    np.save('m'+str(i)+'.npy',np.array(m[1]))

In [13]:
np.array(m[1])

array([ 5.60776465e+00,  2.40906630e-01, -3.68369057e+00,  7.26632078e+02])

In [14]:
np.max(data_p[:,1])

3741.584

In [15]:
# a=[3.296575094179248, 0.6095424311772768, -6.657664186104298, 818.0481187486655]
# rv1=burr(a[0],a[1],a[2],a[3])

In [16]:
from scipy.stats import burr
f=data_E.iloc[:,21:31]
f_e=np.zeros((86,10))
rv1=burr(3.296575094179248, 0.6095424311772768, -6.657664186104298, 818.0481187486655)
rv2=burr(2.96278230336626, 0.7060818040736765, 0.03808196123981357, 617.1479208748744)
rv3=burr(3.8794797413534, 0.39063634718617846, -3.6147574966601823, 667.2560086831902)
rv4=burr(5.607764646791989, 0.24090663005789292, -3.6836905712303682, 726.6320778736202)
x=np.arange(0,4000)
f_e[np.where(f=='0')] = 0
f_e[np.where(f=='1')] = x.dot(rv1.pdf(x))
f_e[np.where(f=='2')] = x.dot(rv2.pdf(x))
f_e[np.where(f=='3')] = x.dot(rv3.pdf(x))
f_e[np.where(f=='4')] = x.dot(rv4.pdf(x))

In [17]:

f=data_E.iloc[:,21:31]
f_e=np.zeros((86,10))
rv1=burr(m1[0], m1[1], m1[2], m1[3])
rv2=burr(m2[0], m2[1], m2[2], m2[3])
rv3=burr(m3[0], m3[1], m3[2], m3[3])
rv4=burr(m4[0], m4[1], m4[2], m4[3])
x=np.arange(0,4000)
f_e[np.where(f=='0')] = 0
f_e[np.where(f=='1')] = x.dot(rv1.pdf(x))
f_e[np.where(f=='2')] = x.dot(rv2.pdf(x))
f_e[np.where(f=='3')] = x.dot(rv3.pdf(x))
f_e[np.where(f=='4')] = x.dot(rv4.pdf(x))

NameError: name 'm1' is not defined

In [ ]:
np.where(f=='0')

In [ ]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(np.hstack((np.int8(f),f_e)), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.hstack((np.int8(f),f_e)))))
r2_score(tend, lin_reg.predict(np.hstack((np.int8(f),f_e))))


In [ ]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(np.int8(f), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.int8(f))))
r2_score(tend, lin_reg.predict(np.int8(f)))

In [ ]:
w=lin_reg.coef_
f3=np.sum(np.int8(f)*w,axis=1)
w

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(np.int8(f), tend)
w1=lin_reg.coef_
np.save('w1.npy',w1)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(np.int8(f_e), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.int8(f_e))))
r2_score(tend, lin_reg.predict(np.int8(f_e)))


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(np.int8(f_e), tend)
w=lin_reg.coef_
np.save('w.npy',w)

In [ ]:
w=lin_reg.coef_
f2=np.sum(np.int8(f_e)*w,axis=1)

In [ ]:
w

In [ ]:
w=np.array([-0.00605373,  0.00757651, -0.01513362, -0.00152401,  0.01459809,
        0.01201937, -0.02436635, -0.01537651, -0.02237698, -0.01520981])
f2=np.sum(np.int8(f_e)*w,axis=1)

In [ ]:
0.        , -1.91508998,  0.66987016,  1.23326549,  0.652249  ,
       -0.58347982

In [ ]:
x1=stress
x2=tired
f1=-1.91508998*np.int8(x1)+0.66987016*np.int8(x2)+1.23326549*np.int8(x1)**2+0.652249*np.int8(x1)*np.int8(x2)-0.58347982*np.int8(x2)**2

In [ ]:
tend.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((f1.reshape(86,1),f2.reshape(86,1))), tend, test_size=0.2, random_state=0)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(y_test, lin_reg.predict(X_test)))
r2_score(y_test, lin_reg.predict(X_test))

In [ ]:
print(mean_absolute_error(tend, lin_reg.predict(np.hstack((f1.reshape(86,1),f2.reshape(86,1))))))
r2_score(tend, lin_reg.predict(np.hstack((f1.reshape(86,1),f2.reshape(86,1)))))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.hstack((f1.reshape(86,1),f2.reshape(86,1),f3.reshape(86,1))), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(y_test, lin_reg.predict(X_test)))
r2_score(y_test, lin_reg.predict(X_test))
print(mean_absolute_error(tend, lin_reg.predict(np.hstack((f1.reshape(86,1),f2.reshape(86,1),f3.reshape(86,1))))))
r2_score(tend, lin_reg.predict(np.hstack((f1.reshape(86,1),f2.reshape(86,1),f3.reshape(86,1)))))

In [ ]:
data_E.iloc[:,21:33].shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_E.iloc[:,21:33], tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(data_E.iloc[:,21:33])))
r2_score(tend, lin_reg.predict(data_E.iloc[:,21:33]))

In [ ]:
risk=data_E.iloc[:,34]
delay=data_E.iloc[:,39]
X_train, X_test, y_train, y_test = train_test_split(np.vstack((risk,delay)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((risk,delay)).T)))
r2_score(tend, lin_reg.predict(np.vstack((risk,delay)).T))

In [ ]:
risk=data_E.iloc[:,34]
delay=data_E.iloc[:,39]
m=[risk=='1']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp1 = [i for i in a[0] if i in b[0]]
#print(np.mean(tend[tmp1]))
k11=np.mean(tend[tmp1])
#print(np.median(tend[tmp1]))
k12=np.median(tend[tmp1])
counts = np.bincount(tend[tmp1])
#print(np.argmax(counts))
k13=np.argmax(counts)
m=[risk=='2']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp2 = [i for i in a[0] if i in b[0]]
tend[tmp2]
#plt.hist(tend[tmp2])
#print(np.mean(tend[tmp2]))
k21=np.mean(tend[tmp2])
#print(np.median(tend[tmp2]))
k22=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
#print(np.argmax(counts))
k23=np.argmax(counts)
m=[risk=='1']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp3 = [i for i in a[0] if i in b[0]]
tend[tmp3]
#plt.hist(tend[tmp2])
#print(np.mean(tend[tmp2]))
k31=np.mean(tend[tmp2])
#print(np.median(tend[tmp2]))
k32=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
#print(np.argmax(counts))
k33=np.argmax(counts)
m=[risk=='2']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp4 = [i for i in a[0] if i in b[0]]
tend[tmp4]
#plt.hist(tend[tmp2])
#print(np.mean(tend[tmp2]))
k41=np.mean(tend[tmp2])
#print(np.median(tend[tmp2]))
k42=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
#print(np.argmax(counts))
k43=np.argmax(counts)
k=np.zeros((4,3))
k[0,0]=k11
k[1,0]=k21
k[2,0]=k31
k[3,0]=k41
k[0,1]=k12
k[1,1]=k22
k[2,1]=k32
k[3,1]=k42
k[0,2]=k13
k[1,2]=k23
k[2,2]=k33
k[3,2]=k43

In [ ]:
m=[risk=='1']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp1 = [i for i in a[0] if i in b[0]]
print(np.mean(tend[tmp1]))
k11=np.mean(tend[tmp1])
print(np.median(tend[tmp1]))
k12=np.median(tend[tmp1])
counts = np.bincount(tend[tmp1])
print(np.argmax(counts))
k13=np.argmax(counts)

In [ ]:
m=[risk=='2']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp2 = [i for i in a[0] if i in b[0]]
tend[tmp2]
#plt.hist(tend[tmp2])
print(np.mean(tend[tmp2]))
k21=np.mean(tend[tmp2])
print(np.median(tend[tmp2]))
k22=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
print(np.argmax(counts))
k23=np.argmax(counts)

In [ ]:
m=[risk=='1']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp3 = [i for i in a[0] if i in b[0]]
tend[tmp3]
#plt.hist(tend[tmp2])
print(np.mean(tend[tmp2]))
k31=np.mean(tend[tmp2])
print(np.median(tend[tmp2]))
k32=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
print(np.argmax(counts))
k33=np.argmax(counts)

In [ ]:
m=[risk=='2']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp4 = [i for i in a[0] if i in b[0]]
tend[tmp4]
#plt.hist(tend[tmp2])
print(np.mean(tend[tmp2]))
k41=np.mean(tend[tmp2])
print(np.median(tend[tmp2]))
k42=np.median(tend[tmp2])
counts = np.bincount(tend[tmp2])
print(np.argmax(counts))
k43=np.argmax(counts)

In [ ]:
k=np.zeros((4,3))
k[0,0]=k11
k[1,0]=k21
k[2,0]=k31
k[3,0]=k41
k[0,1]=k12
k[1,1]=k22
k[2,1]=k32
k[3,1]=k42
k[0,2]=k13
k[1,2]=k23
k[2,2]=k33
k[3,2]=k43
k

In [ ]:
k=np.array([[18.83333333, 20.5       , 21.        ],
        [16.95      , 16.        , 16.        ],
        [17.7826087 , 17.        , 13.        ],
        [17.7826087 , 17.        , 13.        ]])

In [ ]:
f41=np.zeros(86)
f41[tmp1]=k11
f41[tmp2]=k21
f41[tmp3]=k31
f41[tmp3]=k41
f42=np.zeros(86)
f42[tmp1]=k12
f42[tmp2]=k22
f42[tmp3]=k32
f42[tmp3]=k42
f43=np.zeros(86)
f43[tmp1]=k13
f43[tmp2]=k23
f43[tmp3]=k33
f43[tmp3]=k43
X_train, X_test, y_train, y_test = train_test_split(f41.reshape(-1,1), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(f41.reshape(-1,1))))
print(r2_score(tend, lin_reg.predict(f41.reshape(-1,1))))
X_train, X_test, y_train, y_test = train_test_split(f42.reshape(-1,1), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(f42.reshape(-1,1))))
print(r2_score(tend, lin_reg.predict(f42.reshape(-1,1))))
X_train, X_test, y_train, y_test = train_test_split(f43.reshape(-1,1), tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(f43.reshape(-1,1))))
print(r2_score(tend, lin_reg.predict(f43.reshape(-1,1))))
X_train, X_test, y_train, y_test = train_test_split(np.vstack((f42,f41,f43)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f42,f41,f43)).T)))
print(r2_score(tend, lin_reg.predict(np.vstack((f42,f41,f43)).T)))
w=lin_reg.coef_
f4=np.sum(np.vstack((f42,f41,f43)).T*w,axis=1)
w

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(np.vstack((f42,f41,f43)).T, tend)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.vstack((f1,f2,f3,f4)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4)).T))

In [ ]:
# polynomial weight
w0=np.array([ 0.        , -1.91508998,  0.66987016,  1.23326549,  0.652249  ,
       -0.58347982])
# f_e weight (familarity_exploration)
w=np.array([-0.00605373,  0.00757651, -0.01513362, -0.00152401,  0.01459809,
        0.01201937, -0.02436635, -0.01537651, -0.02237698, -0.01520981])
# f weight (familarity_exploration)
w1=np.array([-1.20174752, -2.69788972, -0.01272427, -1.23121747, -2.15756708,
       -0.26401346, -2.54347086,  4.44088608,  1.18509329,  0.26091479])
# risk and delay
w2=np.array([-0.8100222 ,  0.23049743,  0.64396399])
np.save('w0.npy',w0)
np.save('w.npy',w)
np.save('w1.npy',w1)
np.save('w2.npy',w2)

In [ ]:
k=np.array([[18.83333333, 20.5       , 21.        ],
       [16.95      , 16.        , 16.        ],
       [17.7826087 , 17.        , 13.        ],
       [17.7826087 , 17.        , 13.        ]])
np.save('k.npy',k)

In [ ]:
from factors import gen_f
gen_f(data_E,update=False)

In [ ]:
f1,f2,f3,f4=gen_f(data_E,update=False)
X_train, X_test, y_train, y_test = train_test_split(np.vstack((f1,f2,f3,f4)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4)).T))

In [ ]:
time=np.float32(data_E.iloc[:,1:11])
X_train, X_test, y_train, y_test = train_test_split(time, tend, test_size=0.2, random_state=0)
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(time)))
r2_score(tend, lin_reg.predict(time))

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(time, tend)
w_t=lin_reg.coef_
np.save('w_t.npy',w_t)

In [ ]:
w_t=lin_reg.coef_
np.save('w_t.npy',w_t)

In [ ]:
f5=np.sum(time*w_t,axis=1)
X_train, X_test, y_train, y_test = train_test_split(f5.reshape(-1,1), tend, test_size=0.2, random_state=0)
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(f5.reshape(-1,1))))
r2_score(tend, lin_reg.predict(f5.reshape(-1,1)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.vstack((f1,f2,f3,f4,f5)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.vstack((f1,f3,f4,f5)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f3,f4,f5)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f3,f4,f5)).T))

In [ ]:
from g_factors import gen_f
from sklearn.model_selection import train_test_split 
f1,f2,f3,f4,f5=gen_f(data_E,update=False)

X_train, X_test, y_train, y_test = train_test_split(np.vstack((f1,f2,f3,f4,f5)).T, tend, test_size=0.2, random_state=0)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T))

In [ ]:
data_E.iloc[:,34].shape

In [ ]:
test=np.hstack((data_E.iloc[:,21:33],np.array(data_E.iloc[:,34]).reshape(20,1),np.array(data_E.iloc[:,39]).reshape(20,1),data_E.iloc[:,1:11]))

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(mean_absolute_error(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T)))
r2_score(tend, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T))

In [ ]:
choice3_t=data_T.iloc[:,37]
choice4_t=data_T.iloc[:,38]
tend_t=np.int8(choice3_t)+np.int8(choice4_t)
tend_t.shape

In [ ]:
f2.shape

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from g_factors import gen_f
f1,f2,f3,f4,f5=gen_f(data_T,update=False)
choice3_t=data_T.iloc[:,37]
choice4_t=data_T.iloc[:,38]
tend_t=np.int8(choice3_t)+np.int8(choice4_t)
tend_t.shape
print(mean_absolute_error(tend_t, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T)))
r2_score(tend_t, lin_reg.predict(np.vstack((f1,f2,f3,f4,f5)).T))

In [ ]:
m1=np.load('m1.npy')
m2=np.load('m2.npy')
m3=np.load('m3.npy')
m4=np.load('m4.npy')
w=np.load('w.npy')
from scipy.stats import burr
f=data_T.iloc[:,21:31]
f_e=np.zeros((data_T.shape[0],10))
rv1=burr(m1[0], m1[1], m1[2], m1[3])
rv2=burr(m2[0], m2[1], m2[2], m2[3])
rv3=burr(m3[0], m3[1], m3[2], m3[3])
rv4=burr(m4[0], m4[1], m4[2], m4[3])
x=np.arange(0,4000)
f_e[np.where(f=='0')] = 0
f_e[np.where(f=='1')] = x.dot(rv1.pdf(x))
f_e[np.where(f=='2')] = x.dot(rv2.pdf(x))
f_e[np.where(f=='3')] = x.dot(rv3.pdf(x))
f_e[np.where(f=='4')] = x.dot(rv4.pdf(x))
f2=np.sum(np.int8(f_e)*w,axis=1)

In [ ]:
f2.shape

In [ ]:
data_E.shape[0]

In [ ]:
from scipy.stats import burr
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from distribut import best_fit_distribution
import numpy as np
update=False
data_E=data_T
# polynomial weight
w0=np.load('w0.npy')
# f_e weight (familarity_exploration)
w=np.load('w.npy')
# f weight (familarity_exploration)
w1=np.load('w1.npy')
# risk and delay
k=np.load('k.npy')
w2=np.load('w2.npy')
#time
w_t=np.load('w_t.npy')
# burr
m1=np.load('m1.npy')
m2=np.load('m2.npy')
m3=np.load('m3.npy')
m4=np.load('m4.npy')
if update==1:
    np.save('w0_old.npy',w0)
    np.save('w_old.npy',w)
    np.save('w1_old.npy',w1)
    np.save('w2_old.npy',w2)
    np.save('k_old.npy',k)
    np.save('w_t_old.npy',w_t)
    m1=np.save('m1.npy',m1)
    m2=np.save('m2.npy',m2)
    m3=np.save('m3.npy',m3)
    m4=np.save('m4.npy',m4)
    model_update(data_E)
stress=np.int8(data_E.iloc[:,31])
tired=np.int8(data_E.iloc[:,32])
st=np.int8(data_E.iloc[:,31:33])
choice3=data_E.iloc[:,37]
choice4=data_E.iloc[:,38]
tend=np.int8(choice3)+np.int8(choice4)
x1=stress
x2=tired
f1=w0[0]+w0[1]*np.int8(x1)+w0[2]*np.int8(x2)+w0[3]*np.int8(x1)**2+w0[4]*np.int8(x1)*np.int8(x2)+w0[5]*np.int8(x2)**2
f=data_E.iloc[:,21:31]
f_e=np.zeros((data_E.shape[0],10))
rv1=burr(m1[0], m1[1], m1[2], m1[3])
rv2=burr(m2[0], m2[1], m2[2], m2[3])
rv3=burr(m3[0], m3[1], m3[2], m3[3])
rv4=burr(m4[0], m4[1], m4[2], m4[3])
x=np.arange(0,4000)
f_e[np.where(f=='0')] = 0
f_e[np.where(f=='1')] = x.dot(rv1.pdf(x))
f_e[np.where(f=='2')] = x.dot(rv2.pdf(x))
f_e[np.where(f=='3')] = x.dot(rv3.pdf(x))
f_e[np.where(f=='4')] = x.dot(rv4.pdf(x))
f2=np.sum(np.int8(f_e)*w,axis=1)
f3=np.sum(np.int8(f)*w1,axis=1)

risk=data_E.iloc[:,34]
delay=data_E.iloc[:,39]
m=[risk=='1']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp1 = [i for i in a[0] if i in b[0]]
m=[risk=='2']
n=[delay=='1']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp2 = [i for i in a[0] if i in b[0]]
m=[risk=='1']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp3 = [i for i in a[0] if i in b[0]]
m=[risk=='2']
n=[delay=='2']
a=np.array(np.where(np.array(m[0])==1))
b=np.array(np.where(np.array(n[0])==1))
tmp4 = [i for i in a[0] if i in b[0]]
tend[tmp4]
f41=np.zeros(data_E.shape[0])
f41[tmp1]=k[0,0]
f41[tmp2]=k[1,0]
f41[tmp3]=k[2,0]
f41[tmp3]=k[3,0]
f42=np.zeros(data_E.shape[0])
f42[tmp1]=k[0,1]
f42[tmp2]=k[1,1]
f42[tmp3]=k[2,1]
f42[tmp3]=k[3,1]
f43=np.zeros(data_E.shape[0])
f43[tmp1]=k[0,2]
f43[tmp2]=k[1,2]
f43[tmp3]=k[2,2]
f43[tmp3]=k[3,2]
f4=np.sum(np.vstack((f42,f41,f43)).T*w2,axis=1)
time=np.float32(data_E.iloc[:,1:11])
f5=np.sum(time*w_t,axis=1)

In [ ]:
f2